In [1]:
from itertools import combinations
from functools import reduce
from pyspark.sql.functions import col, when, concat, concat_ws, collect_list
from pyspark.sql.types import StringType
import random
from sklearn.cluster import KMeans
import numpy as np

In [2]:
traces =(spark.read.option('mergeSchema', 'true')
              .parquet('data/14-11-18*.parquet'))

In [3]:
frontend = 'web-service_HomeControllerHome_avg_dur'

backends = [c for c in traces.columns
        if c !='traceId' and c.endswith('avg_self_dur')]

In [4]:
frontendSLA = traces.approxQuantile([frontend], [0.99],0)[0][0]

In [5]:
maxs ={ c : traces.select(c).rdd.max()[0]
        for c in backends }
mins ={ c : traces.select(c).rdd.min()[0]
        for c in backends }

In [6]:
normalizedTrace = reduce(lambda df, c : df.withColumn(c, (col(c) - mins[c])/(maxs[c] - mins[c])),
                        backends,
                        traces)

In [7]:
from sklearn.metrics import precision_recall_curve, roc_curve

thresholdsDict, fprDict, tprDict = {}, {}, {}
y = [1 if row[0]>frontendSLA else 0
     for row in traces.select(frontend).collect()]

for aBackend in backends:
    scores = [row[0] for row in normalizedTrace.select(aBackend).collect()]
    fpr, tpr, thresholds = roc_curve(y, scores)
    thresholdsDict[aBackend] = thresholds[:0:-1]
    fprDict[aBackend] = [float(fpr_) for fpr_ in fpr[:0:-1]]
    tprDict[aBackend] = [float(tpr_) for tpr_ in tpr[:0:-1]]

In [8]:
from itertools import combinations
from functools import reduce
from pyspark.sql.functions import col, when, concat, concat_ws, collect_list
from pyspark.sql.types import StringType
import random
from gurobipy import Model, GRB


class ThresholdsSelector:
    def __init__(self, thresholds,
                 tpr, fpr, k):
        self.thresholds = thresholds
        self.k = k
        self.tpr = tpr
        self.fpr = fpr
        self.initModel()
        
    def initModel(self):
        self.m = Model("m")
        self.createVars()
        self.addConstrs()
        self.setObj()
    
    def createVars(self):
        m, n, k = self.m, len(self.thresholds) - 1, self.k
        self.x = m.addVars(n, k, vtype=GRB.BINARY, name="x")
        self.zt = m.addVars(k, vtype=GRB.CONTINUOUS, name="zt" )
        self.zf = m.addVars(k, vtype=GRB.CONTINUOUS, name="zf" )
        self.yt = m.addVars(k, vtype=GRB.CONTINUOUS, name="yt" )
        self.yf = m.addVars(k, vtype=GRB.CONTINUOUS, name="yf" )
        self.ytmax = m.addVar(vtype=GRB.CONTINUOUS, name='ytmax')
        self.yfmax = m.addVar(vtype=GRB.CONTINUOUS, name='yfmax')
        self.ytmin = m.addVar(vtype=GRB.CONTINUOUS, name='ytmin')
        self.yfmin = m.addVar(vtype=GRB.CONTINUOUS, name='yfmin')
        self.st = m.addVar(vtype=GRB.CONTINUOUS, name="st")
        self.sf = m.addVar(vtype=GRB.CONTINUOUS, name="sf")
        
    def addConstrs(self):
        self.addXConstrs()
        self.addZConstrs()
        self.addYConstrs()
        self.addYMaxMinConstrs()
        self.addSConstrs()
        
    def addXConstrs(self):
        n = len(self.thresholds) -1
        self.m.addConstrs((self.x.sum('*',i) == 1
                      for i in range(self.k)), 'c1')
        self.m.addConstrs((self.x.sum(i,'*') <= 1
                      for i in range(n)), 'c2')
        
    def addZConstrs(self):
        for i in range(self.k):
            coeffTP, coeffFP = self.createCoeffs(i)
            self.m.addConstr(self.x.prod(coeffTP, '*',i) == self.zt[i],
                             'c3.%d'% i)
            self.m.addConstr(self.x.prod(coeffFP, '*',i) == self.zf[i],
                             'c4.%d'% i)
            if i==0:
                self.m.addConstr(self.tpr[i] >= self.zt[i],
                                 'c5.%d' % i)
                self.m.addConstr(self.fpr[i] >= self.zf[i],
                                 'c6.%d' % i) 
            else:
                self.m.addConstr(self.zt[i-1]>= self.zt[i],
                                 'c5.%d' % i)
                self.m.addConstr(self.zf[i-1]>= self.zf[i],
                                 'c6.%d' % i)
    def addYConstrs(self):
        self.m.addConstr(self.tpr[0] - self.zt[0] == self.yt[0], 'c7.0')
        self.m.addConstr(self.fpr[0] - self.zf[0] == self.yf[0], 'c8.0')
        for i in range(1,self.k):
            self.m.addConstr(self.zt[i-1] - self.zt[i] == self.yt[i], 'c7.%d' % i)
            self.m.addConstr(self.zf[i-1] - self.zf[i] == self.yf[i], 'c8.%d' % i) 
        
    def addYMaxMinConstrs(self):
        self.m.addGenConstrMax(self.ytmax, self.yt, name='c.9')
        self.m.addGenConstrMax(self.yfmax, self.yf, name='c.10')
        self.m.addGenConstrMin(self.ytmin, self.yt, name='c.11')
        self.m.addGenConstrMin(self.yfmin, self.yf, name='c.12') 
        
    def addSConstrs(self):
        self.m.addConstr( self.zt[self.k-1] - self.tpr[-1] == self.st, 'c13')
        self.m.addConstr(self.zf[self.k-1] - self.fpr[-1]  == self.sf, 'c14')
        
    def createCoeffs(self,i):
        tpr, fpr = self.tpr[1:], self.fpr[1:]
        coeffTP = {(j,i): tpr_ for j, tpr_ in  enumerate(tpr)}
        coeffFP = {(j,i): fpr_ for j, fpr_ in  enumerate(fpr)}
        return coeffTP, coeffFP
    
    def setObj(self):
        obj = (self.sf - self.yfmin) + (self.yfmax - self.yfmin) + (self.ytmax - self.ytmin) + (self.st - self.ytmin)
        self.m.setObjective(obj, GRB.MINIMIZE)
    
    def compute(self):
        self.m.optimize()
        selectedThresholds = [0]
        for i,j in self.x:
            if self.x[i,j].X ==1:
                selectedThresholds.append(self.thresholds[i+1])
        return selectedThresholds

In [9]:
class CacheMaker:
    def __init__(self, traces, backends,
            frontend, frontendSLA):
        self.traces = traces
        self.frontend =frontend
        self.backends = backends
        self.frontendSLA = frontendSLA
    
    def tpCol(self, index):
        return 'tp-%d'% index
    
    def fpCol(self, index):
        return 'fp-%d'% index

    def withColsTpFp(self, df, enumThreshold, aBackend):
        index, threshold = enumThreshold
        aboveThresholdCond = col(aBackend)>= threshold
        tpCond = aboveThresholdCond & (col(self.frontend) > self.frontendSLA)
        fpCond = aboveThresholdCond & (col(self.frontend) <= self.frontendSLA)
        whenTrueOneElseZero = lambda cond : when(cond, '1').otherwise('0')
        return (df.withColumn(self.tpCol(index), whenTrueOneElseZero(tpCond))
                  .withColumn(self.fpCol(index), whenTrueOneElseZero(fpCond)))
    
    def createDfWithColsTpFp(self, enumThresholds, aBackend):
        return reduce(lambda df, enumThr: self.withColsTpFp(df, enumThr, aBackend),
                      enumThresholds,
                      self.traces)
    
    def stringifyCol(self, column):
        colToList = collect_list(col(column))
        listToString = concat_ws("", colToList)
        return listToString.alias(column)
    
    def genStringifyCols(self,enumThresholds):
        for i, _  in enumThresholds:
            yield self.stringifyCol(self.tpCol(i))
            yield self.stringifyCol(self.fpCol(i))
    
    def addRowToCache(self, cache, row, aBackend):
        for i in range(len(row)//2):
            tpBitString = row[self.tpCol(i)]
            fpBitString = row[self.fpCol(i)]
            cache[aBackend, i] = int(tpBitString, 2),int(fpBitString, 2)
    
    def createBitStringsRow(self, thresholdsDict, aBackend):
        enumThresholds = list(enumerate(thresholdsDict[aBackend]))
        df = self.createDfWithColsTpFp(enumThresholds, aBackend)                
        stringifyCols = list(self.genStringifyCols(enumThresholds))
        return (df.groupBy()
                 .agg(*stringifyCols)
                 .collect()[0])
    
    def create(self, thresholdsDict):
        cache = {}
        for aBackend in backends:
            row = self.createBitStringsRow(thresholdsDict, aBackend)
            self.addRowToCache(cache, row, aBackend)
        return cache

In [10]:
import random, copy
from deap import base, creator, tools, algorithms

class FitnessUtils:
    def __init__(self, backends , cache):
        self.backends = backends
        self.cache = cache
        self.p = bin(self.getTPBitString(self.backends[0], 0)).count('1')
    
    def countOnesInConjunctedBitStrings(self, ind, getter):
        
        bit = reduce(lambda bits,bt : bits & getter(*bt),
                     zip(self.backends[1:], ind[1:]),
                     getter(backends[0], ind[0]))
        return bin(bit).count("1")
    
    def getTPBitString(self, backend, threshold):
        return self.cache[backend, threshold][0]
    
    def getFPBitString(self, backend, threshold):
        return self.cache[backend, threshold][1]
    
    def computeTP(self, ind):
        getter = lambda backend, threshold : self.getTPBitString(backend,threshold)
        return self.countOnesInConjunctedBitStrings(ind, getter)
        
    def computeFP(self, ind):
        getter = lambda backend, threshold : self.getFPBitString(backend,threshold)
        return self.countOnesInConjunctedBitStrings(ind, getter)
        
    def computePrecRec(self, ind):
        tp = self.computeTP(ind)
        fp = self.computeFP(ind)
        prec = tp/(tp + fp) if (tp + fp) > 0 else 0
        rec= tp/self.p
        return prec, rec
    
    def computeFMeasure(self, ind):
        prec, rec = self.computePrecRec(ind)
        return 2*(prec*rec)/(prec+rec) if prec > 0 or rec > 0 else 0            

In [43]:
class GA:
    def __init__(self, backends, thresholdSizes, cache):
        self.backends = backends
        self.thresholdSizes = thresholdSizes
        self.fitnessUtils = FitnessUtils(backends, cache)
        self.initGA()
    
    def initGA(self):
        creator.create("Fitness", base.Fitness, weights=(1.0,))
        creator.create("Individual", list, fitness=creator.Fitness)
        self.toolbox = base.Toolbox()
        self.registerAttributes()
        self.registerIndividual()
        self.registerPop()
        self.registerMateMutateAndSelect()
        self.registerEvaluate()
        
    def registerAttributes(self):
        for b in self.backends:
            self.toolbox.register(b, lambda x, y: 0, 0, self.thresholdSizes[b]-1)
    
    def registerIndividual(self):
        attrs = (self.toolbox.__dict__[b] for b in self.backends)
        self.toolbox.register("individual",
                 tools.initCycle,
                 creator.Individual,
                 tuple(attrs))
    def registerPop(self):
        self.toolbox.register("population",
                              tools.initRepeat,
                              list,
                              self.toolbox.individual)

    def registerMateMutateAndSelect(self):
        self.toolbox.register("mate", tools.cxTwoPoint)
        self.toolbox.register("mutate",
                         tools.mutUniformInt,
                         low = [0]*len(self.backends),
                         up = [self.thresholdSizes[b] - 1 for b in self.backends],
                         indpb = 1.0/len(self.backends))
        self.toolbox.register("select", tools.selTournament, tournsize=3)
    
    def registerEvaluate(self):
        self.toolbox.register("evaluate",
                         lambda ind: (self.fitnessUtils.computeFMeasure(ind),))
        
        
    def compute(self, popSize=100, maxGen=400, mutProb=0.2):
        self.toolbox.pop_size = popSize
        self.toolbox.max_gen = maxGen
        self.toolbox.mut_prob = mutProb
        pop = self.toolbox.population(n=self.toolbox.pop_size)
        pop = self.toolbox.select(pop, len(pop))
        return algorithms.eaMuPlusLambda(pop, self.toolbox, mu=self.toolbox.pop_size, 
                                         lambda_=self.toolbox.pop_size, 
                                         cxpb=1-self.toolbox.mut_prob,
                                         mutpb=self.toolbox.mut_prob, 
                                         stats=None, 
                                         ngen=self.toolbox.max_gen, 
                                         verbose=None)[0]

In [44]:
from random import randint

def createRdmThresholdsDict(thresholdsDict, k):
    rdmThresholdsDict = {}
    for aBackend in backends:
        thresholds = thresholdsDict[aBackend]
        if k + 1 >= (len(thresholds)):
            rdmThresholdsDict[aBackend] = thresholds
            continue
        selected = [thresholds[0]]
        while len(selected) < k:
            i = randint(1, len(thresholds)-1)
            if not thresholds[i] in selected:
                selected.append(thresholds[i])
        rdmThresholdsDict[aBackend] = sorted(selected)
    return rdmThresholdsDict

In [45]:
def createBalancedThresholdDict(thresholdsDict, tprDict, fprDict, k):
    balThresholdsDict = {}
    for aBackend in backends:
        thresholds = thresholdsDict[aBackend]
        if k + 1 >= (len(thresholds)):
            balThresholdsDict[aBackend] = thresholds
            continue
        tpr = tprDict[aBackend]
        fpr = fprDict[aBackend]
        selected = ThresholdsSelector(thresholds, tpr, fpr,k).compute()
        balThresholdsDict[aBackend] = selected
    return balThresholdsDict

In [46]:
def createClustThresholdDict(thresholdsDict, tprDict, fprDict, k):
    clustThresholdDict = {}
    for aBackend in backends:
        thresholds = thresholdsDict[aBackend]
        if k + 1 >= (len(thresholds)):
            clustThresholdDict[aBackend] = thresholds
            continue
        tpr = tprDict[aBackend]
        fpr = fprDict[aBackend]
        X = list(zip(tpr[1:], fpr[1:]))
        distances = KMeans(n_clusters=k, random_state=0).fit_transform(X)
        indices = [ np.argsort(distances[:,i])[0] for i in range(k)]
        clustThresholdDict[aBackend] =[thresholds[0]] + sorted(thresholds[i+1] for i in indices)
    return clustThresholdDict

In [47]:
k = 50
rdmThresholdsDict = createRdmThresholdsDict(thresholdsDict, k)
#balThresholdsDict = createBalancedThresholdDict(thresholdsDict, tprDict, fprDict, k)
clustThresholdsDict = createClustThresholdDict(thresholdsDict, tprDict, fprDict, k)

In [48]:
res = []
for thrsDict in [rdmThresholdsDict, clustThresholdsDict]:
    cache = CacheMaker(normalizedTrace, backends,frontend, frontendSLA).create(thrsDict)
    ga = GA(backends,  { b:len(thrsDict[b]) for b in backends } , cache)
    res.append(ga.compute())

/home/luca/environments/spark/lib/python3.6/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'Fitness' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
/home/luca/environments/spark/lib/python3.6/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


In [49]:
def denormalize(ind):
    return [(thresholdsDict[b][t] *(maxs[b] - mins[b]) + mins[b], b.split('_')[1].replace('avg_self_dur', ''))
            for t, b in zip(ind, backends) if t>0]

In [50]:
solutions = sorted([(ga.fitnessUtils.computeFMeasure(ind), *ga.fitnessUtils.computePrecRec(ind), denormalize(ind)) for ind in res[0]],
                    key=lambda x:x[0])

for f, prec, rec, sol in solutions:
    print (sol)
    print('F-score: ',f)
    print('Precision:',prec, '\t Recall: ', rec, '\n\n')

[(1100.0, 'get'), (1400.0, 'ItemsControllerFinditemsrandombyidproduct'), (1000.0, 'ProductsControllerFindproduct'), (179438.0, 'HomeControllerHome')]
F-score:  0.31847133757961776
Precision: 0.4768392370572207 	 Recall:  0.2390710382513661 


[(1100.0, 'get'), (1400.0, 'ItemsControllerFinditemsrandombyidproduct'), (1000.0, 'ProductsControllerFindproduct'), (179438.0, 'HomeControllerHome')]
F-score:  0.31847133757961776
Precision: 0.4768392370572207 	 Recall:  0.2390710382513661 


[(1100.0, 'get'), (1400.0, 'ItemsControllerFinditemsrandombyidproduct'), (1000.0, 'ProductsControllerFindproduct'), (179438.0, 'HomeControllerHome')]
F-score:  0.31847133757961776
Precision: 0.4768392370572207 	 Recall:  0.2390710382513661 


[(1100.0, 'get'), (1400.0, 'ItemsControllerFinditemsrandombyidproduct'), (1000.0, 'ProductsControllerFindproduct'), (179438.0, 'HomeControllerHome')]
F-score:  0.31847133757961776
Precision: 0.4768392370572207 	 Recall:  0.2390710382513661 


[(1100.0, 'get'), (1400.0, '

In [51]:
solutions = sorted([(ga.fitnessUtils.computeFMeasure(ind), *ga.fitnessUtils.computePrecRec(ind), denormalize(ind)) for ind in res[1]],
                    key=lambda x:x[0])

for f, prec, rec, sol in solutions:
    print (sol)
    print('F-score: ',f)
    print('Precision:',prec, '\t Recall: ', rec, '\n\n')

[(1400.0, 'get'), (1400.0, 'ItemsControllerFinditemsrandombyidproduct'), (1000.0, 'ProductsControllerFindproduct'), (179202.0, 'HomeControllerHome')]
F-score:  0.5038639876352395
Precision: 0.4044665012406948 	 Recall:  0.6680327868852459 


[(1400.0, 'get'), (1400.0, 'ItemsControllerFinditemsrandombyidproduct'), (1000.0, 'ProductsControllerFindproduct'), (179202.0, 'HomeControllerHome')]
F-score:  0.5038639876352395
Precision: 0.4044665012406948 	 Recall:  0.6680327868852459 


[(1400.0, 'get'), (1400.0, 'ItemsControllerFinditemsrandombyidproduct'), (1000.0, 'ProductsControllerFindproduct'), (179202.0, 'HomeControllerHome')]
F-score:  0.5038639876352395
Precision: 0.4044665012406948 	 Recall:  0.6680327868852459 


[(1400.0, 'get'), (1400.0, 'ItemsControllerFinditemsrandombyidproduct'), (1000.0, 'ProductsControllerFindproduct'), (179202.0, 'HomeControllerHome')]
F-score:  0.5038639876352395
Precision: 0.4044665012406948 	 Recall:  0.6680327868852459 


[(1400.0, 'get'), (1400.0, 'Item

Precision: 0.4044665012406948 	 Recall:  0.6680327868852459 


[(1400.0, 'get'), (1400.0, 'ItemsControllerFinditemsrandombyidproduct'), (1000.0, 'ProductsControllerFindproduct'), (179202.0, 'HomeControllerHome')]
F-score:  0.5038639876352395
Precision: 0.4044665012406948 	 Recall:  0.6680327868852459 


[(1400.0, 'get'), (1400.0, 'ItemsControllerFinditemsrandombyidproduct'), (1000.0, 'ProductsControllerFindproduct'), (179202.0, 'HomeControllerHome')]
F-score:  0.5038639876352395
Precision: 0.4044665012406948 	 Recall:  0.6680327868852459 


[(1400.0, 'get'), (1400.0, 'ItemsControllerFinditemsrandombyidproduct'), (1000.0, 'ProductsControllerFindproduct'), (179202.0, 'HomeControllerHome')]
F-score:  0.5038639876352395
Precision: 0.4044665012406948 	 Recall:  0.6680327868852459 


[(1400.0, 'get'), (1400.0, 'ItemsControllerFinditemsrandombyidproduct'), (1000.0, 'ProductsControllerFindproduct'), (179202.0, 'HomeControllerHome')]
F-score:  0.5038639876352395
Precision: 0.4044665012406948 	 